In [1]:
from keras.models import Model, load_model, model_from_json
import keras
from keras.datasets import cifar10
from keras import regularizers, callbacks
import sys
sys.path.append('C:/Users/320060820/experiments/')
from train_model import *
from model_gen.inception_model import get_model
import copy

import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import math

from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

# to restart layer numbering
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [2]:
# dataset
import scipy.io as sio

train_data = sio.loadmat('../datasets/svhn/train_32x32.mat')
test_data = sio.loadmat('../datasets/svhn/test_32x32.mat')

(x_train, y_train), (x_test, y_test) = (train_data['X'], train_data['y']), (test_data['X'], test_data['y'])

y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

x_train = x_train[np.newaxis,...]
x_train = np.swapaxes(x_train,0,4).squeeze()

x_test = x_test[np.newaxis,...]
x_test = np.swapaxes(x_test,0,4).squeeze()

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [3]:
train_size = x_train.shape[0]
test_size = x_test.shape[0]

x_train_resize = np.zeros(shape = (train_size, 96, 96, 3))
x_test_resize = np.zeros(shape = (test_size, 96, 96, 3))

In [4]:
for i in range(0, train_size):
    x_train_resize[i, :, :, :] = resize(x_train[i, :, :, :], (96, 96, 3))
for i in range(0, test_size):
    x_test_resize[i, :, :, :]=resize(x_test[i, :, :, :], (96, 96, 3))

x_train_resize = np.reshape(x_train_resize, (len(x_train_resize), 96, 96, 3))  
x_test_resize = np.reshape(x_test_resize, (len(x_test_resize), 96, 96, 3))

In [5]:
model = get_model((96, 96, 3), 10, nb_fabrics=49, scale_factor=96//96, resampling='conv', pool_layer='max', dropout=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 48, 48, 20)   560         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 48, 48, 20)   80          conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 48, 48, 16)   336         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_norm

In [7]:
train_data, test_data = (x_train_resize, y_train), (x_test_resize, y_test)
train_data, test_data, trained_model, history = train_model(
                    model, train_data, test_data, 30, 
                    64, 10, 1, rgb=True)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 73257 samples, validate on 26032 samples
Epoch 1/30
73257/73257 [==============================] - 142s 2ms/step - loss: 2.5436 - acc: 0.1923 - val_loss: 1.8700 - val_acc: 0.3433
Epoch 2/30
73257/73257 [==============================] - 136s 2ms/step - loss: 1.4215 - acc: 0.5218 - val_loss: 0.9771 - val_acc: 0.6825
Epoch 3/30
73257/73257 [==============================] - 137s 2ms/step - loss: 0.9350 - acc: 0.6974 - val_loss: 0.7035 - val_acc: 0.7776
Epoch 4/30
73257/73257 [==============================] - 136s 2ms/step - loss: 0.7393 - acc: 0.7650 - val_loss: 0.4768 - val_acc: 0.8513
Epoch 5/30
73257/73257 [==============================] - 136s 2ms/step - loss: 0.6356 - acc: 0.8006 - val_loss: 0.4495 - val_acc: 0.8636
Epoch 6/30
73257/73257 [==============================] - 137s 2ms/step - loss: 0.5697 - acc: 0.8228 - val_loss: 0.4523 - val_acc: 0.8

In [8]:
trained_model.save('../test_models/svhn_df/svhn_df.h5')
trained_model.save_weights('../test_models/svhn_df/svhn_df_w.h5')